**Antes de começar**

In [ ]:
# Este código garante que o notebook sempre vai buscar os módulos
%load_ext autoreload
%autoreload 2


# Aula de Processamento de Dados

## O que vamos fazer?

Criar um pacote em python que vai nos ajudar a tratar e processar os dados do **Titanic**

Mas antes, vamos entender o que é um **pacote para Python**:

👉 Código reutilizável de um projeto para outro (de... importar...)

Um pacote permite que você:

👉 Compartilhe com outras pessoas

- Instalar a partir do PyPI: pip install <nome_do_pacote>
- Instalar a partir do GitHub: pip install git+https://...

👉 Implantar em produção (em servidores Linux)

👉 Rastreie o código (git) e colabore nele!

🎯 Objetivo da aula: criar um pacote chamado `intelidata` que você possa instalar em qualquer máquina

`pip instalar intelidata`

## Qual a diferença entre módulo e pacote?

- Um módulo é um único arquivo python dentro de um pacote
- Um pacote é um diretório de módulos python que contém um __init__.py

Então vamos criar uma pasta chamada `intelidata` e colocar o arquivo `__init__.py` e `lib.py`

1. O arquivo `__init__.py` tem duas funções principais em um pacote Python:
    
    * **Marcação de Diretório**: Primeiro e mais fundamental, a presença de um arquivo `__init__.py` em um diretório sinaliza para o Python que esse diretório deve ser tratado como um pacote ou subpacote. Isso permite que você faça importações usando o nome do diretório. Por exemplo, se você tem um diretório `mypackage` com um arquivo `__init__.py` dentro dele, você pode importar módulos desse diretório usando `import mypackage.mymodule`.
        
    * **Execução de Código de Inicialização**: Quando o pacote é importado, o código no arquivo `__init__.py` é executado automaticamente. Isso é útil para qualquer configuração ou inicialização que você queira fazer para o seu pacote. Por exemplo, você pode inicializar algumas variáveis, importar submódulos ou executar qualquer código de configuração necessário.

2. O arquivo `lib.py` é onde vai ser codificado as funções para que o pacote funcione.

In [ ]:
!mkdir intelidata
!touch intelidata/__init__.py
!touch intelidata/lib.py


#############################################################################

Vamos incluir no arquivo `lib.py` o seguinte código:

```python
def preprocess():
    print("Vamos processar os dados!")

if __name__ == '__main__':
    preprocess()
```
#############################################################################

Agora criamos o arquivo de setup para que o comando `pip install` funcione. O setup.py é um arquivo tradicionalmente usado na distribuição e instalação de bibliotecas e aplicativos Python. Ele contém informações sobre o pacote que você deseja distribuir e instruções sobre como instalá-lo. Esse arquivo é utilizado principalmente com a ferramenta setuptools, que é uma biblioteca para facilitar a distribuição de pacotes Python.

In [ ]:
!touch setup.py


Preenchemos com o seguinte código:

```python
# setup.py
from setuptools import setup

setup(name='intelidata',
      description="este pacote instala os preprocessadores",
      packages=["intelidata"]) # Aqui podemos ter vários pacotes...
```

E então instalamos com `pip install .`

In [ ]:
!pip install .


Processing /content
  Preparing metadata (setup.py) ... done
  Created wheel for intelidata: filename=intelidata-0.0.0-py3-none-any.whl size=1411 sha256=d751d122aac5bcd96efda449a37f2bcaeffe51d28e8b9ccabf4643a4e82c0124
  Stored in directory: /tmp/pip-ephem-wheel-cache-afaj_7q0/wheels/e8/d3/96/0e8c7135806cbda4db28d12fc8d710e5e4f66ced1411163e67
Successfully built intelidata


Agora podemos chamar a função que tem no arquivo `lib.py`

In [ ]:
from intelidata.lib import preprocess

preprocess()


Vamos processar os dados!


Agora vamos criar uma CLI com o Makefile

In [ ]:
!touch Makefile


O Makefile é muito sensível, então não deixe de passar este código com o espaçamento em `tab`:

```makefile
install:
    @pip install -e .
```

Note que o `install:` é o comando, e desta vez passamos a instalação com o `-e`, é usado para instalar um pacote Python em modo "editável" ou "desenvolvimento". Neste momento vamos utilizar desta forma pois ele guarda caches do nosso desenvolvimento. Quando for para produção lembre-se de mudar!

In [ ]:
!make install


Makefile:2: *** missing separator.  Stop.


In [ ]:
# O comando tree mostra como está a estrutura de diretórios,
# isso é bom para a documentação por exemplo...
!tree


/bin/bash: line 1: tree: command not found


Agora vamos criar uma pasta de teste para o nosso pacote,, para isso criamos um arquivo chamado `requirements.txt` com as bibliotecas que somos dependentes, e uma dessas vai ser o `pytest`

In [ ]:
!touch requirements.txt
!echo pytest >> requirements.txt
!pip install -e .
!pytest tests -v # verbose


Obtaining file:///content
  Preparing metadata (setup.py) ... done
  Attempting uninstall: intelidata
    Found existing installation: intelidata 0.0.0
    Uninstalling intelidata-0.0.0:
      Successfully uninstalled intelidata-0.0.0
  Running setup.py develop for intelidata
======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.2, pluggy-1.3.0 -- /usr/bin/python3
cachedir: .pytest_cache
rootdir: /content
plugins: anyio-3.7.1
collected 0 items                                                                                  

====================================== no tests ran in 0.00s =======================================
ERROR: file or directory not found: tests



Legal! temos o pacote de testes, mas não temos nenhum teste, vamos criar agora a pasta `tests` e o nosso primeiro teste

In [ ]:
!mkdir tests
!touch tests/test_lib.py


Coloque este código dentro do `test_lib.py`:

```python
import pytest
from intelidata.lib import preprocess

def test_preprocess_output(capfd):  # capfd é um "fixture" do pytest para capturar saídas impressas.
    preprocess()
    out, err = capfd.readouterr()
    assert out == "Vamos processar os dados!\n", "A saída impressa não corresponde ao esperado"
```

**Opa**, temos que fazer com que o setup leia o requirements!

```python
from setuptools import setup

# Lê o arquivo requirements.txt e coloca as dependências em uma lista
with open('requirements.txt') as f:
    required = f.read().splitlines()

setup(
    name='intelidata',
    description='este pacote instala os preprocessadores',
    packages=['intelidata'],
    install_requires=required  # Aqui as dependências são passadas
)
```

In [ ]:
!pytest tests

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.2, pluggy-1.3.0
rootdir: /content
plugins: anyio-3.7.1
collected 1 item                                                                                   

tests/test_lib.py .                                                                          [100%]

======================================== 1 passed in 0.01s =========================================


No Makefile coloque o comando de testes:

```makefile
# Makefile
test:
  @pytest -v tests
```

e rodamos o comando de teste:

In [ ]:
!make test


### Agora vamos criar o load dos dados

Vamos adicionar o código:

```python
def load_data():
    # Ler o arquivo CSV usando pandas
    df = pd.read_csv("titanic_dataset.csv")

    return df
```

no arquivo `lib.py`

In [ ]:
# Fazemos o load data
import intelidata.lib as id

df = id.load_data()


In [ ]:
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Vemos que o dataset está cheio de furos de dados, vamos criar uma função para limpar os dados no `lib.py`:

```python
def clean_data(df: pd.DataFrame) -> pd.DataFrame:
    # Removendo linhas que contêm valores NaN
    cleaned_df = df.dropna()

    return cleaned_df
```

e rodamos:

In [ ]:
from intelidata.lib import clean_data

df = clean_data(df)


In [ ]:
df


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
10,11,1,3,"Sandstrom, Miss. Marguerite Rut",female,4.0,1,1,PP 9549,16.7000,G6,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
...,...,...,...,...,...,...,...,...,...,...,...,...
871,872,1,1,"Beckwith, Mrs. Richard Leonard (Sallie Monypeny)",female,47.0,1,1,11751,52.5542,D35,S
872,873,0,1,"Carlsson, Mr. Frans Olof",male,33.0,0,0,695,5.0000,B51 B53 B55,S
879,880,1,1,"Potter, Mrs. Thomas Jr (Lily Alexenia Wilson)",female,56.0,0,1,11767,83.1583,C50,C
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S


E os testes? vamos atualizar os testes:

```python
import intelidata.lib as intelidata
import pandas as pd

def test_preprocess_output(capfd):  # capfd é um "fixture" do pytest para capturar saídas impressas.
    intelidata.preprocess()
    out, err = capfd.readouterr()
    assert out == "Vamos processar os dados!\n", "A saída impressa não corresponde ao esperado"

def test_load_data():
    df = intelidata.load_data()
    assert isinstance(df, pd.DataFrame), "A função não retorna um DataFrame do pandas."
    # Supondo que o arquivo "titanic_dataset.csv" tenha pelo menos uma linha (sem contar o cabeçalho)
    assert not df.empty, "O DataFrame retornado está vazio."

def test_clean_data():
    # Criando um DataFrame de exemplo com valores NaN
    df = pd.DataFrame({
        'A': [1, 2, np.nan],
        'B': [4, np.nan, 6],
        'C': [np.nan, 8, 9],
    })
    
    cleaned_df = intelidata.clean_data(df)
    assert cleaned_df.shape[0] == 1, "O DataFrame limpo deve conter apenas uma linha."
    assert cleaned_df.shape[1] == 3, "O DataFrame limpo deve conter três colunas."
    assert not cleaned_df.isnull().any().any(), "O DataFrame limpo não deve conter valores NaN."
```

In [ ]:
!make test


# Extra!

Podemos configurar uma função para apenas enviar ao S3 da AWS por exemplo...

```python
import pandas as pd
import boto3
from io import BytesIO

def send_to_s3(df: pd.DataFrame, bucket_name: str, file_name: str, aws_access_key: str, aws_secret_key: str):
    """
    Envia um DataFrame pandas para um bucket S3 da Amazon.

    Parâmetros:
    - df (pd.DataFrame): DataFrame a ser enviado.
    - bucket_name (str): Nome do bucket S3.
    - file_name (str): Nome do arquivo no S3.
    - aws_access_key (str): AWS Access Key.
    - aws_secret_key (str): AWS Secret Key.

    Retorna:
    - None
    """
    # Inicializar o cliente S3
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key, aws_secret_access_key=aws_secret_key)
    
    # Converter o DataFrame para CSV e depois para Bytes
    csv_buffer = BytesIO()
    df.to_csv(csv_buffer)
    
    # Enviar os bytes para o S3
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())
```

neste caso, temos que colocar o boto3 no requirements.txt, assim como criar o teste também. Agora fica com você!